In [ ]:
#The following code compares 2 csv files using vectorization. Each file contains a column that can be used as a unique row identifier.
#The unique identifier column will be used to compare the rows with the same ID between the 2 files (see "Key" value below).
#The resulting output (updated_df) contains 1 row from first file, followed by the corresponding row from the 2nd file(same ID) and a 3rd row containing True/False
#corresponding to matching/different values. The 3 rows sequence repeats for all content.
#The second resulting output is extra_records_df, which contains extra rows that do not have a match in the other file. See column "Exist".

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [14]:
#Read csv files and store results in dataframes
df1 = pd.read_csv('.//Path1/report1.csv', sep = ';', header = None)
df2 = pd.read_csv('.//Path2/report2.csv', sep = ';', header = None)

#Reset index
df1 = df1.reset_index(drop = True)
df2 = df2.reset_index(drop = True)

#Merge dataframes and add indicator column to show row origin
df_diff = pd.merge(df1, df2, how = 'outer', indicator = 'Exist')

#Filter dataframe to show only rows with differences or extra rows
df_diff = df_diff.query("Exist != 'both'")

In [15]:
#Rename the unique identifier column to Key and use it to sort the dataframe
df_diff = df_diff.rename(columns = {2: 'Key'})
df_diff = df_diff.sort_values(['Key'])

In [17]:
#Create a list of all "Key" values without a pair
key_counts = df_diff['Key'].value_counts()
single_keys = key_counts[key_counts == 1].index.tolist()

#Generate the list of extra records in both files. Check column "Exist"
extra_records_df = df_diff[df_diff['Key'].isin(single_keys)]

In [18]:
extra_records_df

,0,1,Key,3,4,5,6,7,8,9,Exist
16,10y,sensitivity,ccyL,1000,1500,2300,2333,5000.0,9000,12000,right_only
17,10y,sensitivity,ccyM,1000,1500,2300,2333,5000.0,9000,12000,right_only
7,10y,sensitivity,ccyZ,1000,1500,2300,2333,5000.0,9000,12000,left_only


In [19]:
#Generate the records with "Key" pairs
df_diff = df_diff[-df_diff['Key'].isin(single_keys)]

In [ ]:
#Convert all columns to object type (string). This returns warnings but works ok for now.
cols = df_diff.columns
df_diff[cols] = df_diff[cols].astype('str')

#Create a new dataframe with shifted df_diff values
shifted_df = df_diff.shift(-1)

#Create a boolean mask for rows where the Key column is equal to the next row
mask = df_diff['Key'] == shifted_df['Key']

#Initialize an empty dataframe with the same columns as the original one
result_df = pd.DataFrame(columns = df_diff.columns)

In [39]:
#Record the processing start time
start_time = time.time()

#Iterate over columns, excluding Key, compare values and store True/False in the results dataframe
for col in df_diff.columns[1:]:
    result_df[col] = np.where(mask, df_diff[col] == shifted_df[col], False)

#Add the Key column to the result dataframe
result_df['Key'] = np.where(mask, df_diff['Key'], np.nan)

#Drop all rows in the result dataframe where Key is NaN, as they are not relevant
result_df = result_df.dropna(subset = ['Key'])

#Record the end processing time and display duration
end_time = time.time()
processing_duration = end_time - start_time
minutes, seconds = divmod(processing_duration, 60)
print(f'Processing duration: {int(minutes)} m {seconds:.2f} s')

Processing duration: 0 m 0.01 s


In [54]:
#Reset index, remove current "Exist" columns
df_diff = df_diff.reset_index(drop = True)
result_df = result_df.reset_index(drop = True)
df_diff = df_diff.drop('Exist', axis = 1)
result_df = result_df.drop('Exist', axis = 1)

In [64]:
#Merge df_diff and results_df, format results
updated_df = pd.merge(df_diff, result_df, how = 'outer', indicator = 'Exist')
updated_df = updated_df.drop('Exist', axis = 1)
updated_df = updated_df.sort_values(['Key'])
updated_df.style.apply(lambda x: ['background-color: orange' if v == False else '' for v in x], axis=0)

,0,1,Key,3,4,5,6,7,8,9
0,1m,sensitivity,ccyA,1000,1500,2300,2333,5000.0,9000,12000
1,1m,sensitivityy,ccyA,1000,1500,2300,2333,5000.0,9000,12000
8,nan,False,ccyA,True,True,True,True,True,True,True
2,1y,sensitivity,ccyF,1000,1500,2300,2333,5000.0,9000,12001
3,1y,sensitivity,ccyF,9991000,1500,2300,2333,5000.0,9000,12001
9,nan,True,ccyF,False,True,True,True,True,True,True
4,3y,sensitivity,ccyH,1000,1500,2300,2333,5000.0,9000,12000
5,3y,sensitivity,ccyH,1000,1500,2300,2333,9995000.0,9000,12000
10,nan,True,ccyH,True,True,True,True,False,True,True
6,7y,sensitivity,ccyI,1000,1500,2300,2333,5000.0,9000,12000


In [ ]:
#Save to Excel if needed:
updated_df.to_excel('diff_excel.xlsx', index = False)

In [49]:
result_df

,0,1,Key,3,4,5,6,7,8,9
0,NaN,False,ccyA,True,True,True,True,True,True,True
1,NaN,True,ccyF,False,True,True,True,True,True,True
2,NaN,True,ccyH,True,True,True,True,False,True,True
3,NaN,True,ccyI,True,True,True,True,False,True,True


In [50]:
df_diff

,0,1,Key,3,4,5,6,7,8,9
0,1m,sensitivity,ccyA,1000,1500,2300,2333,5000.0,9000,12000
1,1m,sensitivityy,ccyA,1000,1500,2300,2333,5000.0,9000,12000
2,1y,sensitivity,ccyF,1000,1500,2300,2333,5000.0,9000,12001
3,1y,sensitivity,ccyF,9991000,1500,2300,2333,5000.0,9000,12001
4,3y,sensitivity,ccyH,1000,1500,2300,2333,5000.0,9000,12000
5,3y,sensitivity,ccyH,1000,1500,2300,2333,9995000.0,9000,12000
6,7y,sensitivity,ccyI,1000,1500,2300,2333,5000.0,9000,12000
7,7y,sensitivity,ccyI,1000,1500,2300,2333,nan,9000,12000


In [62]:
updated_df

,0,1,Key,3,4,5,6,7,8,9
0,1m,sensitivity,ccyA,1000,1500,2300,2333,5000.0,9000,12000
1,1m,sensitivityy,ccyA,1000,1500,2300,2333,5000.0,9000,12000
8,NaN,False,ccyA,True,True,True,True,True,True,True
2,1y,sensitivity,ccyF,1000,1500,2300,2333,5000.0,9000,12001
3,1y,sensitivity,ccyF,9991000,1500,2300,2333,5000.0,9000,12001
9,NaN,True,ccyF,False,True,True,True,True,True,True
4,3y,sensitivity,ccyH,1000,1500,2300,2333,5000.0,9000,12000
5,3y,sensitivity,ccyH,1000,1500,2300,2333,9995000.0,9000,12000
10,NaN,True,ccyH,True,True,True,True,False,True,True
6,7y,sensitivity,ccyI,1000,1500,2300,2333,5000.0,9000,12000
